In [2]:
import dask.dataframe as dd

# Merge events data for labels
train_series = dd.read_parquet('../data/train_series_engineered_dask.parquet')
print(f"train_series head \n{train_series.head()}")
train_events = dd.read_csv('../data/train_events.csv')
train_events['timestamp'] = dd.to_datetime(train_events['timestamp'], format='%Y-%m-%dT%H:%M:%S%z')

train_data = train_series.merge(train_events, on=['series_id', 'step'], how='left')
print(f"train_data head \n{train_data.head()}")
train_data['event'] = train_data['event'].fillna('no_event')  # Fill NaN with 'no_event'



# Categorize 'event' column
train_data = train_data.categorize(columns=['event'])

event_encoded = train_data['event'].to_dask_array(lengths=True)
event_encoded = event_encoded.codes  # This gives you the encoded labels

# Assign back to Dask DataFrame
train_data['event_encoded'] = event_encoded



train_series head 
      series_id  step           timestamp  anglez    enmo timezone  \
0  038441c925bb     0 2018-08-14 15:30:00  2.6367  0.0217    -0400   
1  038441c925bb     1 2018-08-14 15:30:05  2.6368  0.0215    -0400   
2  038441c925bb     2 2018-08-14 15:30:10  2.6370  0.0216    -0400   
3  038441c925bb     3 2018-08-14 15:30:15  2.6368  0.0213    -0400   
4  038441c925bb     4 2018-08-14 15:30:20  2.6368  0.0215    -0400   

   anglez_rolling_mean_5  enmo_rolling_mean_5  anglez_rolling_std_5  \
0                    NaN                  NaN                   NaN   
1                    NaN                  NaN                   NaN   
2                    NaN                  NaN                   NaN   
3                    NaN                  NaN                   NaN   
4                2.63682              0.02152               0.00011   

   enmo_rolling_std_5  ...  anglez_rolling_std_30  enmo_rolling_std_30  \
0                 NaN  ...                    NaN          

c:\Users\dekel\AppData\Local\Programs\Python\Python310\lib\site-packages\dask\dataframe\core.py:7256: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise a warning unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df = func(*args, **kwargs)


train_data head 
      series_id  step         timestamp_x  anglez    enmo timezone  \
0  038441c925bb     0 2018-08-14 15:30:00  2.6367  0.0217    -0400   
1  038441c925bb     1 2018-08-14 15:30:05  2.6368  0.0215    -0400   
2  038441c925bb     2 2018-08-14 15:30:10  2.6370  0.0216    -0400   
3  038441c925bb     3 2018-08-14 15:30:15  2.6368  0.0213    -0400   
4  038441c925bb     4 2018-08-14 15:30:20  2.6368  0.0215    -0400   

   anglez_rolling_mean_5  enmo_rolling_mean_5  anglez_rolling_std_5  \
0                    NaN                  NaN                   NaN   
1                    NaN                  NaN                   NaN   
2                    NaN                  NaN                   NaN   
3                    NaN                  NaN                   NaN   
4                2.63682              0.02152               0.00011   

   enmo_rolling_std_5  ...  enmo_rolling_mean_60  anglez_rolling_std_60  \
0                 NaN  ...                   NaN            

c:\Users\dekel\AppData\Local\Programs\Python\Python310\lib\site-packages\dask\dataframe\core.py:7256: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise a warning unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df = func(*args, **kwargs)
c:\Users\dekel\AppData\Local\Programs\Python\Python310\lib\site-packages\dask\dataframe\core.py:7256: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise a warning unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df = func(*args, **kwargs)
c:\Users\dekel\AppData\Local\Programs\Python\Python310\lib\site-packages\dask\dataframe\core.py:

AttributeError: 'Array' object has no attribute 'codes'

In [ ]:
from sklearn.model_selection import train_test_split

# X = train_data.drop(['event', 'event_encoded', 'timestamp', 'series_id'], axis=1)
# y = train_data['event_encoded']

# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

import numpy as np

# Function to assign each row to either training or validation set
def assign_train_test(test_size=0.2):
    return 'train' if np.random.rand() > test_size else 'test'

# Adding a new column 'set' to the DataFrame
train_data['set'] = train_data.map_partitions(lambda df: df.apply(lambda row: assign_train_test(row), axis=1))

# Separating features and labels
X_train = train_data[train_data['set'] == 'train'].drop(['event', 'event_encoded', 'timestamp', 'series_id', 'set'], axis=1)
y_train = train_data[train_data['set'] == 'train']['event_encoded']

X_val = train_data[train_data['set'] == 'test'].drop(['event', 'event_encoded', 'timestamp', 'series_id', 'set'], axis=1)
y_val = train_data[train_data['set'] == 'test']['event_encoded']


In [ ]:
# from sklearn.ensemble import RandomForestClassifier

# clf = RandomForestClassifier(random_state=42)
# clf.fit(X_train, y_train)

from dask_ml.xgboost import XGBClassifier

clf = XGBClassifier()
clf.fit(X_train, y_train)



In [ ]:
from sklearn.metrics import classification_report

y_pred = clf.predict(X_val)
print(classification_report(y_val, y_pred, target_names=le.classes_))
